In [2]:
#Importo le librerie necessarie
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [3]:
#Memorizzo all'interno di soup tutto il codice HTML del sito scrollabile sotto forma di oggetto soup

# Initialize WebDriver
driver = webdriver.Chrome()  # You should replace this with the appropriate web driver you're using (Chrome, Firefox, etc.)

# Open the URL of the webpage
url = "https://www.kammi.it/scarpe-uomo/"
driver.get(url)

# Automatically scroll the page
scroll_pause_time = 2  # Pause between each scroll
screen_height = driver.execute_script("return window.screen.height;")  # Browser window height
i = 1
while True:
    # Scroll down
    driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
    i += 1
    time.sleep(scroll_pause_time)

    # Check if reaching the end of the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    if screen_height * i > scroll_height:
        break

# Fetch the data using BeautifulSoup after all data is loaded
soup = BeautifulSoup(driver.page_source, "html.parser")
# Process and save the data as needed

# Close the WebDriver session
driver.quit()

In [4]:
#Creo una lista contenente tutti i blocchi di dato presenti nella pagina web principale
blocks=soup.find_all("div",class_="col-6 col-md-4 col-lg-3 col-xl-auto mt-4")

In [5]:
#Estrazione lista completa dei link alle pagine web accessibili a partire dalla pagina web principale, contenenti ciacuna le info di un prodotto
k=0
links=[]
for block in blocks:
    for child in block:
        if child=="\n":
            continue
        for neph in child.children:         
            if neph=="\n" or neph.name=="script":
                continue
            site=neph.get("href")
            links.append(site)
print(len(links))            
links
    

31


['https://www.kammi.it/scarpe-uomo/sneakers-1057-idp21567.html',
 'https://www.kammi.it/scarpe-uomo/sneakers-1057-idp21566.html',
 'https://www.kammi.it/scarpe-uomo/derby-810-idp21565.html',
 'https://www.kammi.it/scarpe-uomo/derby-7308-idp21564.html',
 'https://www.kammi.it/scarpe-uomo/derby-7308-idp21563.html',
 'https://www.kammi.it/scarpe-uomo/sneakers-1009-idp21547.html',
 'https://www.kammi.it/scarpe-uomo/sneakers-5947-idp21546.html',
 'https://www.kammi.it/scarpe-uomo/sneakers-4652-idp21539.html',
 'https://www.kammi.it/scarpe-uomo/sneakers-3499-idp21538.html',
 'https://www.kammi.it/scarpe-uomo/allacciata-0600-idp21540.html',
 'https://www.kammi.it/scarpe-uomo/allacciata-0600-idp18396.html',
 'https://www.kammi.it/scarpe-uomo/allacciata-320-idp18394.html',
 'https://www.kammi.it/scarpe-uomo/allacciata-320-idp7734.html',
 'https://www.kammi.it/scarpe-uomo/stivaletto-1352-idp21493.html',
 'https://www.kammi.it/scarpe-uomo/stivaletto-1352-idp21490.html',
 'https://www.kammi.it/sca

In [6]:
#creazione dataframe KAMMI 
product_info={}
products_info=[]
k=0
all_sizes=list(range(35,55))
for link in links:
    k=k+1
    print(k)
    req=requests.get(link)
    soup=BeautifulSoup(req.text)
    product_name=soup.find_all("h1",class_="text-black h4 fw-bold")[0].text
    description=soup.find_all("p",class_="mt-4")[0].text
    materials=soup.find_all("p",class_="mt-4")[1].text.split()
    if soup.find_all("div",class_="option-color selected")!=[]:
        color_element_sel=soup.find_all("div",class_="option-color selected")[0]
        color_sel=color_element_sel.find("div").find("span").find("div").get("style").rstrip().rstrip(";").strip("background").lstrip(":")
    if soup.find_all("img",class_="mditaly2")!= []:
        made_in_italy=1
    else:
        made_in_italy=0
    #break
    for i in range(len(materials)):
        if materials[i]=="esterno:":
            esternal_material=materials[i+1]
        elif materials[i]=="interno:":
            internal_material=materials[i+1]
        elif materials[i]=="suola:":
            sole_material=materials[i+1]
    price=soup.find_all("p",class_="mt-2")[0].text
    product_info["product_name"]=product_name
    product_info["description"]=description
    product_info["color"]=color_sel
    product_info["made_in_italy"]=made_in_italy
    product_info["esternal_material"]=esternal_material
    product_info["internal_material"]=internal_material
    product_info["sole_material"]=sole_material
    product_info["price"]=price
    size_elements=soup.find_all("button",class_="content d-flex align-items-center justify-content-center p-2 checkRitiroPP")
    available_sizes=[]
    for size in size_elements:
        size_number=size.text.strip()
        product_info[str(size_number)]=1
        available_sizes.append(size_number)
    not_available_sizes=[size for size in all_sizes if size not in available_sizes]
    for size in not_available_sizes:
        product_info[str(size)]=0
    products_info.append(product_info)
    product_info={}
    
products_info
#materials
df_Kammi=pd.DataFrame(products_info)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [7]:
df_Kammi

product_name                                        description  \
0        Sneakers 1057  Sneakers in pelle verdone con inserti color se...   
1        Sneakers 1057  Sneakers in pelle bianca con inserti in camosc...   
2            Derby 810  Derby liscia in camoscio sigaro con fondo blu ...   
3           Derby 7308  Derby in camoscio sabbia con lavorazione a cod...   
4           Derby 7308  Derby in camoscio oltremare con lavorazione a ...   
5        Sneakers 1009  Sneakers in pelle bianca e beige con inserti d...   
6        Sneakers 5947  Sneakers in pelle bianca con inserti in camosc...   
7        Sneakers 4652  Sneakers in pelle bianca liscia, fondo grezzo....   
8        Sneakers 3499  Sneakers in pelle bianca con inserti in camosc...   
9      Allacciata 0600        Allacciata in abrasivato nero e fondo cuoio   
10     Allacciata 0600  Allacciata in pelle liscia blu notte e fondo c...   
11      Allacciata 320            Allacciata in pelle blu con fondo cuoio   
12      Allacciata 320           Allacciata in pelle nera con fondo cuoio   
13     Stivaletto 1352  Stivaletto stringato modello anfibio nero, sta...   
14     Stivaletto 1352  Stivaletto stringato modello anfibio testa di ...   
15    Allacciata 14350  Allacciata liscia in pelle testa di moro con f...   
16      Mocassino 1366                     Mocassino in pelle color cuoio   
17      Mocassino 1366                            Mocassino in pelle navy   
18      Sneakers Fabio  Sneakers in pelle bianca con microfori e strin...   
19  Sneakers Fabio cam  Sneakers in camoscio sabbia con inserti in pel...   
20  Sneakers Fabio cam  Sneakers in camoscio verde con inserti in pell...   
21     Allacciata 0601                Allacciata classica in vernice nera   
22      Mocassino L011  Mocassino in pelle blu con cuciture a vista e ...   
23      Mocassino L012  Mocassino in pelle testa di moro con cuciture ...   
24      Mocassino 6070  Mocassino in camoscio blu con lacci color test...   
25      Mocassino L010  Mocassino stringato in pelle marrone con lacci...   
26     Mocassino Barca                 Mocassino in pelle ingrassata navy   
27       Mocassino 012                  Moccassino in pelle testa di moro   
28    Allacciata Barca                 Allacciata in pelle ingrassata blu   
29     Mocassino Barca        Mocassino in pelle ingrassata testa di moro   
30    Allacciata Barca       Allacciata in pelle ingrassata testa di moro   

      color  made_in_italy esternal_material internal_material sole_material  \
0   #026f38              1             Pelle             Pelle         Gomma   
1   #FFFFFF              1             Pelle             Pelle         Gomma   
2   #FFFFFF              1             Pelle             Pelle     Sintetico   
3   #ac8e83              1             Pelle             Pelle     Sintetico   
4   #1f2839              1             Pelle             Pelle     Sintetico   
5   #1f2839              0             Pelle             Pelle         Gomma   
6   #1f2839              0             Pelle             Pelle         Gomma   
7   #1f2839              0             Pelle             Pelle         Gomma   
8   #1f2839              0             Pelle             Pelle         Gomma   
9   #000000              1             Pelle             Pelle         Cuoio   
10  #1f2839              1             Pelle             Pelle         Cuoio   
11  #1f2839              1             Pelle             Pelle         Cuoio   
12  #000000              1             Pelle             Pelle         Cuoio   
13  #000000              1             Pelle             Pelle     Sintetico   
14  #786153              1             Pelle             Pelle     Sintetico   
15  #786153              1             Pelle             Pelle     Sintetico   
16  #786153              1             Pelle             Pelle         Cuoio   
17  #1f2839              1             Pelle             Pelle         Cuoio   
18  #1f2839              

In [8]:
df=pd.read_excel("C:/Users/Alessio Mangiagi/Desktop/python_conn/maschera taglie.xlsx")

In [28]:
merged_df = pd.merge(df_Kammi, df, left_index=True, right_index=True, how='right')

In [29]:
merged_df

product_name_x                                      description_x  \
0        Sneakers 1057  Sneakers in pelle verdone con inserti color se...   
1        Sneakers 1057  Sneakers in pelle bianca con inserti in camosc...   
2            Derby 810  Derby liscia in camoscio sigaro con fondo blu ...   
3           Derby 7308  Derby in camoscio sabbia con lavorazione a cod...   
4           Derby 7308  Derby in camoscio oltremare con lavorazione a ...   
5        Sneakers 1009  Sneakers in pelle bianca e beige con inserti d...   
6        Sneakers 5947  Sneakers in pelle bianca con inserti in camosc...   
7        Sneakers 4652  Sneakers in pelle bianca liscia, fondo grezzo....   
8        Sneakers 3499  Sneakers in pelle bianca con inserti in camosc...   
9      Allacciata 0600        Allacciata in abrasivato nero e fondo cuoio   
10     Allacciata 0600  Allacciata in pelle liscia blu notte e fondo c...   
11      Allacciata 320            Allacciata in pelle blu con fondo cuoio   
12      Allacciata 320           Allacciata in pelle nera con fondo cuoio   
13     Stivaletto 1352  Stivaletto stringato modello anfibio nero, sta...   
14     Stivaletto 1352  Stivaletto stringato modello anfibio testa di ...   
15    Allacciata 14350  Allacciata liscia in pelle testa di moro con f...   
16      Mocassino 1366                     Mocassino in pelle color cuoio   
17      Mocassino 1366                            Mocassino in pelle navy   
18      Sneakers Fabio  Sneakers in pelle bianca con microfori e strin...   
19  Sneakers Fabio cam  Sneakers in camoscio sabbia con inserti in pel...   
20  Sneakers Fabio cam  Sneakers in camoscio verde con inserti in pell...   
21     Allacciata 0601                Allacciata classica in vernice nera   
22      Mocassino L011  Mocassino in pelle blu con cuciture a vista e ...   
23      Mocassino L012  Mocassino in pelle testa di moro con cuciture ...   
24      Mocassino 6070  Mocassino in camoscio blu con lacci color test...   
25      Mocassino L010  Mocassino stringato in pelle marrone con lacci...   
26     Mocassino Barca                 Mocassino in pelle ingrassata navy   
27       Mocassino 012                  Moccassino in pelle testa di moro   
28    Allacciata Barca                 Allacciata in pelle ingrassata blu   
29     Mocassino Barca        Mocassino in pelle ingrassata testa di moro   
30    Allacciata Barca       Allacciata in pelle ingrassata testa di moro   

      color  made_in_italy_x esternal_material_x internal_material_x  \
0   #026f38                1               Pelle               Pelle   
1   #FFFFFF                1               Pelle               Pelle   
2   #FFFFFF                1               Pelle               Pelle   
3   #ac8e83                1               Pelle               Pelle   
4   #1f2839                1               Pelle               Pelle   
5   #1f2839                0               Pelle               Pelle   
6   #1f2839                0               Pelle               Pelle   
7   #1f2839                0               Pelle               Pelle   
8   #1f2839                0               Pelle               Pelle   
9   #000000                1               Pelle               Pelle   
10  #1f2839                1               Pelle               Pelle   
11  #1f2839                1               Pelle               Pelle   
12  #000000                1               Pelle               Pelle   
13  #000000                1               Pelle               Pelle   
14  #786153                1               Pelle               Pelle   
15  #786153                1               Pelle               Pelle   
16  #786153                1               Pelle               Pelle   
17  #1f2839                1               Pelle               Pelle   
18  #1f2839                1               Pelle               Pelle   
19  #e4a677                1               Pelle               Pelle   
20  #026f38                1   

In [18]:
import webcolors

In [19]:
def get_color_name(hex_color):
    try:
        color_name = webcolors.hex_to_name(hex_color)
    except ValueError:
        color_name = closest_color(hex_color)
    return color_name

def closest_color(hex_color):
    rgb_color = webcolors.hex_to_rgb(hex_color)
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb_color.red) ** 2
        gd = (g_c - rgb_color.green) ** 2
        bd = (b_c - rgb_color.blue) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

In [20]:
df_Kammi['Color_name'] = df_Kammi['color'].apply(get_color_name)
df_Kammi

product_name                                        description  \
0        Sneakers 1057  Sneakers in pelle verdone con inserti color se...   
1        Sneakers 1057  Sneakers in pelle bianca con inserti in camosc...   
2            Derby 810  Derby liscia in camoscio sigaro con fondo blu ...   
3           Derby 7308  Derby in camoscio sabbia con lavorazione a cod...   
4           Derby 7308  Derby in camoscio oltremare con lavorazione a ...   
5        Sneakers 1009  Sneakers in pelle bianca e beige con inserti d...   
6        Sneakers 5947  Sneakers in pelle bianca con inserti in camosc...   
7        Sneakers 4652  Sneakers in pelle bianca liscia, fondo grezzo....   
8        Sneakers 3499  Sneakers in pelle bianca con inserti in camosc...   
9      Allacciata 0600        Allacciata in abrasivato nero e fondo cuoio   
10     Allacciata 0600  Allacciata in pelle liscia blu notte e fondo c...   
11      Allacciata 320            Allacciata in pelle blu con fondo cuoio   
12      Allacciata 320           Allacciata in pelle nera con fondo cuoio   
13     Stivaletto 1352  Stivaletto stringato modello anfibio nero, sta...   
14     Stivaletto 1352  Stivaletto stringato modello anfibio testa di ...   
15    Allacciata 14350  Allacciata liscia in pelle testa di moro con f...   
16      Mocassino 1366                     Mocassino in pelle color cuoio   
17      Mocassino 1366                            Mocassino in pelle navy   
18      Sneakers Fabio  Sneakers in pelle bianca con microfori e strin...   
19  Sneakers Fabio cam  Sneakers in camoscio sabbia con inserti in pel...   
20  Sneakers Fabio cam  Sneakers in camoscio verde con inserti in pell...   
21     Allacciata 0601                Allacciata classica in vernice nera   
22      Mocassino L011  Mocassino in pelle blu con cuciture a vista e ...   
23      Mocassino L012  Mocassino in pelle testa di moro con cuciture ...   
24      Mocassino 6070  Mocassino in camoscio blu con lacci color test...   
25      Mocassino L010  Mocassino stringato in pelle marrone con lacci...   
26     Mocassino Barca                 Mocassino in pelle ingrassata navy   
27       Mocassino 012                  Moccassino in pelle testa di moro   
28    Allacciata Barca                 Allacciata in pelle ingrassata blu   
29     Mocassino Barca        Mocassino in pelle ingrassata testa di moro   
30    Allacciata Barca       Allacciata in pelle ingrassata testa di moro   

      color  made_in_italy esternal_material internal_material sole_material  \
0   #026f38              1             Pelle             Pelle         Gomma   
1   #FFFFFF              1             Pelle             Pelle         Gomma   
2   #FFFFFF              1             Pelle             Pelle     Sintetico   
3   #ac8e83              1             Pelle             Pelle     Sintetico   
4   #1f2839              1             Pelle             Pelle     Sintetico   
5   #1f2839              0             Pelle             Pelle         Gomma   
6   #1f2839              0             Pelle             Pelle         Gomma   
7   #1f2839              0             Pelle             Pelle         Gomma   
8   #1f2839              0             Pelle             Pelle         Gomma   
9   #000000              1             Pelle             Pelle         Cuoio   
10  #1f2839              1             Pelle             Pelle         Cuoio   
11  #1f2839              1             Pelle             Pelle         Cuoio   
12  #000000              1             Pelle             Pelle         Cuoio   
13  #000000              1             Pelle             Pelle     Sintetico   
14  #786153              1             Pelle             Pelle     Sintetico   
15  #786153              1             Pelle             Pelle     Sintetico   
16  #786153              1             Pelle             Pelle         Cuoio   
17  #1f2839              1             Pelle             Pelle         Cuoio   
18  #1f2839              

In [30]:
df_Kammi.to_excel("kammi-df.xlsx", index=False )